In [29]:
import numpy as np
from collections import deque

# Деревья

- Восстановление бинарного дерева из массива

- Проверка на симметричность бинарного дерева

- Поиск минимальной глубины

- Произведение минимального и максимального значения

- Сравнение двух деревьев

In [30]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [31]:
def print_tree(node, level=0, prefix="Root: "):
    if node is not None:
        print(" " * (4 * level) + prefix + str(node.val))
        print_tree(node.left, level + 1, "L--- ")
        print_tree(node.right, level + 1, "R--- ")

## Восстановление бинарного дерева из массива

Необходимо реализовать функцию, которая будет принимать на вход массив и выстраивать из него бинарное дерево.

In [32]:
# dont skip Nans
def build_tree(arr, i=0):
    if i >= len(arr):
        return None
    node = TreeNode(arr[i])
    node.left = build_tree(arr, 2 * i + 1)
    node.right = build_tree(arr, 2 * i + 2)
    return node

# skip Nans
def build_tree_with_nulls(arr, i=0):
    if i >= len(arr) or arr[i] is None:
        return None
    node = TreeNode(arr[i])
    node.left = build_tree_with_nulls(arr, 2 * i + 1)
    node.right = build_tree_with_nulls(arr, 2 * i + 2)
    return node

In [33]:
arr = [8, 9, 11, 7, 16, 3, 1, 10, 12, 13, 15, 27]
tree = build_tree(arr)
print_tree(tree)

print('-' * 30)

arr = [1, None, 2, None, None, None, 3]
tree = build_tree_with_nulls(arr)
print_tree(tree)

Root: 8
    L--- 9
        L--- 7
            L--- 10
            R--- 12
        R--- 16
            L--- 13
            R--- 15
    R--- 11
        L--- 3
            L--- 27
        R--- 1
------------------------------
Root: 1
    R--- 2
        R--- 3


## Симметричное бинарное дерево
На вход функции подается
бинарное дерево. Необходимо понять, является ли
это дерево симметричным.

### Симметричное бинарное дерево. Breadth-First Search
Обход бинарного дерева в ширину

- Создаем массив узлов дерева, чтобы можно
было по нему итерироваться
- На первой итерации в массиве только корень
- Для каждого уровня создаем очередь из узлов, в
которую будем складывать всех детей текущего
уровня. То есть для 2 уровня (8 и 8) в очереди
будут храниться узлы 9, 6, 6, 9.


In [34]:
def is_symmetric_bfs(root):
    if root == None:
        return True

    queue = deque([root])

    while queue: # текущий уровень для сравнения узлов зеркально друг относительно друга
        level = list(queue)
        n = len(level)

        for i in range(n // 2): # идем от двух концов очереди к середине, сравнивая зеркальные узлы
            left = level[i]
            right = level [n-i-1]

            if left is None and right is None:
                continue
            if left is None or right is None:
                return False
            if left.val != right.val:
                return False

        for _ in range(n): # переход к следующему уровню — добавляем детей всех текущих узлов.
            node = queue.popleft()
            if node:
                queue.append(node.left)
                queue.append(node.right)
    return True

In [35]:
tree = TreeNode(3,
        TreeNode(8, TreeNode(9), TreeNode(6)),
        TreeNode(8, TreeNode(6), TreeNode(9))
    )

print(is_symmetric_bfs(tree))

True


In [36]:
def is_mirror(left, right):
    if left is None and right is None:
        return True
    if left is None or right is None:
        return False
    if left.val != right.val:
        return False
    # рекурсивно сравниваем:
    # левое поддерево "слева" с правым поддеревом "справа"
    # правое поддерево "слева" с левым поддеревом "справа"
    return is_mirror(left.left, right.right) and is_mirror(left.right, right.left)

def is_symmetric(root):
    # Пустое дерево по определению симметрично
    if not root:
        return True
    return is_mirror(root.left, root.right)

In [37]:
root = TreeNode(1,
    left=TreeNode(2, TreeNode(3), TreeNode(4)),
    right=TreeNode(2, TreeNode(4), TreeNode(3))
)

print(is_symmetric(root))

True


### Симметричное дерево. Depth-First Search
Обход дерева в глубину

- Самый простой вариант - это
вариант, при котором при
прохождении по бинарному дереву
поиска получили бы
отсортированный массив LNR.
- [9, 8, 6, 3, 6, 8, 9]


- Если корень пустой, считаем, что дерево
симметрично
- Обходим дерево сохраняя в массив результат
обхода
- Как раньше проверяли очередь - проверяем
массив на симметричность

In [38]:
def is_symmetric_dfs(node, result):
    if not node:
        return

    if node.left and node.right:
        is_symmetric_dfs(node.left.left, result) # рекурсивно уходим в самое левое поддерево
        result.append(node.left.val) # сохраняем значение левого дочернего узла
        is_symmetric_dfs(node.left.right, result) # переходим к правому поддереву левого узла

        result.append(node.val) # добавляем корень поддерева

        is_symmetric_dfs(node.right.left, result) # левое поддерево правого узла
        result.append(node.right.val) # значение правого дочернего узла
        is_symmetric_dfs(node.right.right, result) # переходим к правому поддереву правого узла
    else:
        result.append(node.val)

    return result == result[::-1]

In [39]:
root = TreeNode(1,
    left=TreeNode(2, TreeNode(3), TreeNode(4)),
    right=TreeNode(2, TreeNode(4), TreeNode(3))
)
result = []
print(is_symmetric_dfs(root, result))
print(result)

True
[3, 2, 4, 1, 4, 2, 3]


## Поиск минимальной глубины бинарного дерева
На вход функции подается бинарное
дерево. Необходимо найти минимальную
глубину дерева.


- Если root не существует, значит, на этом уровне
глубина равна нулю
- Если у узла есть и левый, и правый потомки, а
значит, есть и левое, и правое поддеревья,
необходимо вернуть минимальную глубину из
этих поддеревьев
- К результату необходимо прибавить 1, чтобы
учесть корневой уровень
- Если есть только левое/правое поддерево,
продолжаем поиск только по нему, плюс
корневой уровень

In [40]:
def min_depth(root):
    if root is None:
        return 0

    if root.left is None and root.right is None: # листовой узел
        return 1

    if root.left and root.right: # если есть оба потомка, ищем меньший
        return 1 + min(min_depth(root.left), min_depth(root.right))

    if root.left: # только левое поддерево
        return 1 + min_depth(root.left)

    return 1 + min_depth(root.right) # только правое

In [46]:
root = TreeNode(1,
    left=TreeNode(2, TreeNode(3), TreeNode(4)),
    right=TreeNode()
)


print_tree(root)

print(min_depth(root))

Root: 1
    L--- 2
        L--- 3
        R--- 4
    R--- 0
2


## Поиск произведения максимального и минимального элементов

Дано бинарное дерево поиска в виде массива. Необходимо найти
произведение минимального и максимального значений.

- определяем индекс минимального элемента
- устанавливаем min_index равным `1`
- далее в цикле двигаем его на `2 * i + 1`
- определяем индекс максимального элемента
- устанавливаем max_index равным `2`
- далее двигаем его на `2 * i + 2`


In [55]:
def max_min_multiplication(data):
    if len(data) < 3: # недостаточно элементов для потомков
        return -1

    min_index = 1 # первый левый потомок
    max_index = 2 # первый правый потомок

    for i in range(1, len(data)): # проходим по всем левым потомкам, индексы 2 * i + 1
        left = 2 * i + 1
        if left < len(data):
            if data[left] < data[min_index]:
                min_index = left

    for i in range(1, len(data)): # проходим по всем правым потомкам, индексы 2 * i + 2
        right = 2 * i + 2
        if right < len(data):
            if data[right] > data[max_index]:
                max_index = right

    return f'min :{data[min_index]}, max: {data[max_index]}, mult: {data[min_index] * data[max_index]}'

In [56]:
data = [16, 12, 18, 11, 15, 17, 21, 19, 24]
print(max_min_multiplication(data))

min :11, max: 24, mult: 264


## Сравнение двух деревьев
На вход функции подается 2 бинарных
дерева.
Необходимо понять, являются ли эти
два дерева одинаковыми.

- BFS
- DFS (NLR NRL)
- Сравниваем два массива




- Рекурсивный подход
- На каждом вызове сравниваем
соответствующие поддеревья
- Если текущие поддеревья равны
null, значит деревья равны
- Если только одно из поддеревьев
равно null, значит на этом уровне
деревья не равны

In [57]:
def is_same_tree(a, b):
    if a is None and b is None:
        return True

    if a is None or b is None:
        return False

    if a.val != b.val:
        return False

    return is_same_tree(a.left, b.left) and is_same_tree(a.right, b.right)

In [58]:
a = TreeNode(1,
    left=TreeNode(2, TreeNode(3), TreeNode(4)),
    right=TreeNode()
)

b = TreeNode(1,
    left=TreeNode(2, TreeNode(3), TreeNode(4)),
    right=TreeNode(2, TreeNode(3), None)
)

print(is_same_tree(a, b))

False


## Является ли дерево В поддеревом для А
На вход функции подается два бинарных дерева. Необходимо
вернуть true, если дерево B является поддеревом для A

In [59]:
def is_subtree(a, b):
    if b is None:
        return True

    if a is None:
        return False

    if is_same_tree(a, b):
        return True

    return is_subtree(a.left, b) or is_subtree(a.right, b)

In [61]:
a = TreeNode(1,
    left=TreeNode(2, TreeNode(3), TreeNode(4)),
    right=TreeNode()
)

b = TreeNode(1,
    left=TreeNode(2, TreeNode(3), TreeNode(4)),
    right=TreeNode()
)

print(is_same_tree(a, b))

True


## Зеркальные узлы

Дано бинарное дерево. Необходимо подсчитать количество
зеркальных узлов в нем

In [62]:
def dfs(left, right):
    if left is None or right is None: # если один из узлов отсутствует — не пара
        return 0

    count = 0

    if left.val == right.val:  # нашли зеркальную пару по значению
        count = 1

    # продолжаем проверку зеркальных поддеревьев
    count += dfs(left.left, right.right)
    count += dfs(left.right, right.left)

    return count

def count_mirror_twins(root):
    if root is None:
        return 0
    # запускаем зеркальное сравнение левого и правого поддеревьев
    return dfs(root.left, root.right)

In [63]:
root = TreeNode(1,
    left=TreeNode(2, TreeNode(3), TreeNode(4)),
    right=TreeNode(2, TreeNode(4), TreeNode(3))
)

print(count_mirror_twins(root))

3
